
# Week01 — Ollama 기반 Prompt Engineering 실습
**목표:** 로컬 LLM(ollama)을 이용해 3가지 대표 시나리오(요약, Q&A, 스타일 변환)를 실행하고 결과를 기록합니다.

- 환경: Python 3.11.13 (venv 권장)
- 엔진: Ollama (기본 포트 `http://localhost:11434`)
- 모델 예시: `llama3.1:8b` (또는 `mistral:7b`, `qwen2.5:7b` 등)


## 0) 준비물 체크 (버전 확인)

In [14]:

import sys, platform
print("Python:", sys.version)
print("Platform:", platform.platform())


Python: 3.11.9 (main, Apr  2 2024, 08:25:04) [Clang 15.0.0 (clang-1500.3.9.4)]
Platform: macOS-15.6-arm64-arm-64bit


## 1) Ollama 연결 상태 점검

In [15]:

import requests

OLLAMA_HOST = "http://localhost:11434"
try:
    r = requests.get(OLLAMA_HOST)
    print("[OK] Ollama reachable at", OLLAMA_HOST, "| status:", r.status_code)
except Exception as e:
    print("[ERROR] Ollama not reachable at", OLLAMA_HOST)
    print(" - Start Ollama app or service, e.g., 'ollama serve'")
    print(" - Then pull a model: 'ollama pull llama3.1:8b'")
    raise e


[OK] Ollama reachable at http://localhost:11434 | status: 200



## 2) 공용 유틸 함수 — Chat API 호출
- `chat_ollama(model, messages, stream=False)`: /api/chat 호출
- messages 포맷: `[{"role":"system","content":...}, {"role":"user","content":...}]`


In [16]:

import json, time
from typing import List, Dict

def chat_ollama(model: str, messages: List[Dict], stream: bool = False, host: str = OLLAMA_HOST):
    url = f"{host}/api/chat"
    headers = {"Content-Type":"application/json"}
    payload = {
        "model": model,
        "messages": messages,
        "stream": stream
    }
    t0 = time.time()
    resp = requests.post(url, headers=headers, data=json.dumps(payload), timeout=120)
    dt = time.time() - t0
    resp.raise_for_status()
    data = resp.json()
    # Ollama returns {"message": {"content": ...}, "done": true, ...}
    text = data.get("message", {}).get("content", "")
    return text, dt



## 3) 실험 로거 준비 (선택) — pandas CSV 기록


In [17]:

import pandas as pd
from pathlib import Path

LOG_PATH = Path("results.csv")
cols = ["scenario","temperature","top_p","output_len","latency_sec","note"]
if not LOG_PATH.exists():
    pd.DataFrame(columns=cols).to_csv(LOG_PATH, index=False)
print(f"[INFO] Logging to {LOG_PATH.resolve()}")


[INFO] Logging to /Users/sjcha/Documents/3. 아주대AI대학원/2025-2nd-semester/ajou-llmops-2025-2nd-semester/week01/results.csv



## 4) 사례 1 — 텍스트 요약 (Summarization)
- 긴 문단을 2~3문장으로 요약
- system role로 역할/톤 지정


In [ ]:

MODEL = "llama3.1:8b"  # 바꿔도 됨: "mistral:7b", "qwen2.5:7b" 등

text_to_summarize = (
    "Large Language Models are transforming the AI landscape by enabling natural language interfaces "
    "for a wide range of tasks. They can summarize, translate, answer questions, and generate content. "
    "However, they require careful prompting, evaluation, and monitoring to be reliable and cost-effective."
)

messages = [
    {"role":"system","content":"You are a concise and faithful summarizer. Always write 2–3 sentences."},
    {"role":"user","content":f"Summarize the following text in 2–3 sentences:\n\n{text_to_summarize}"}
]

summary, dt = chat_ollama(MODEL, messages)
print(summary, "\n\n[latency_sec]", round(dt, 2))

# log
row = {
    "scenario":"summarization",
    "temperature":None,
    "top_p":None,
    "output_len":len(summary.split()),
    "latency_sec":dt,
    "note":"2-3 sentence summary"
}
pd.concat([pd.read_csv(LOG_PATH), pd.DataFrame([row])]).to_csv(LOG_PATH, index=False)


Large Language Models are revolutionizing the field of artificial intelligence by providing a versatile tool for various natural language processing tasks, but their effectiveness depends on proper usage and management. 

[latency_sec] 4.32


/var/folders/jc/4z1x4d2x08v3pks_cvns5kzr0000gn/T/ipykernel_55350/3893016846.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([pd.read_csv(LOG_PATH), pd.DataFrame([row])]).to_csv(LOG_PATH, index=False)



## 5) 사례 2 — 질의응답 (Q&A)
- 간단한 지식 질문


In [20]:

messages = [
    {"role":"system","content":"You are a knowledgeable assistant for AI courses."},
    {"role":"user","content":"What are the 5 stages of the LLM LifeCycle? Answer in one short paragraph."}
]
answer, dt = chat_ollama(MODEL, messages)
print(answer, "\n\n[latency_sec]", round(dt, 2))

row = {
    "scenario":"qa",
    "temperature":None,
    "top_p":None,
    "output_len":len(answer.split()),
    "latency_sec":dt,
    "note":"LLM LifeCycle question"
}
pd.concat([pd.read_csv(LOG_PATH), pd.DataFrame([row])]).to_csv(LOG_PATH, index=False)


The 5 stages of the Large Language Model (LLM) Lifecycle are:

1. **Training**: The initial stage where the model is trained on a large dataset to learn patterns and relationships.
2. **Evaluation**: The model's performance is assessed using various metrics, such as accuracy and fluency.
3. **Deployment**: The trained model is deployed in a production environment, where it can be fine-tuned for specific tasks or applications.
4. **Maintenance**: The model is continuously monitored and updated to ensure its performance remains optimal over time.
5. **Retirement**: Eventually, the model may become outdated or inefficient, at which point it is replaced with a newer, more advanced version. 

[latency_sec] 12.72


/var/folders/jc/4z1x4d2x08v3pks_cvns5kzr0000gn/T/ipykernel_55350/330923159.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([pd.read_csv(LOG_PATH), pd.DataFrame([row])]).to_csv(LOG_PATH, index=False)



## 6) 사례 3 — 스타일 변환 (Style Transfer)
- 격식 있는 문장을 친근한 톤으로 변환


In [ ]:

source_text = "Dear Professor, I would like to request your feedback on my draft report."
messages = [
    {"role":"system","content":"You rewrite text in a friendly, casual, and polite style."},
    {"role":"user","content":f"Rewrite this in a casual style:\n\n{source_text}"}
]
styled, dt = chat_ollama(MODEL, messages)
print(styled, "\n\n[latency_sec]", round(dt, 2))

row = {
    "scenario":"style_transfer",
    "temperature":None,
    "top_p":None,
    "output_len":len(styled.split()),
    "latency_sec":dt,
    "note":"casual rewrite"
}
pd.concat([pd.read_csv(LOG_PATH), pd.DataFrame([row])]).to_csv(LOG_PATH, index=False)


Here's the rewritten text in a friendly, casual, and polite style:

"Hi Professor, hope you're doing well! I was wondering if you could take a look at my draft report and let me know what you think? Your feedback would be super helpful to me!" 

[latency_sec] 7.22


/var/folders/jc/4z1x4d2x08v3pks_cvns5kzr0000gn/T/ipykernel_55350/3820485004.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  pd.concat([pd.read_csv(LOG_PATH), pd.DataFrame([row])]).to_csv(LOG_PATH, index=False)



## 7) (선택) 파라미터 실험 템플릿
- 일부 로컬 모델은 /api/generate 경로에서 `temperature`, `top_p` 등을 지원합니다.
- 아래는 예시 형태(모델별 파라미터 가용성 상이). 지원되지 않으면 기본값 사용.


In [22]:

def generate_raw(model: str, prompt: str, host: str = OLLAMA_HOST, temperature: float = None, top_p: float = None):
    url = f"{host}/api/generate"
    payload = {"model": model, "prompt": prompt}
    if temperature is not None:
        payload["options"] = payload.get("options", {})
        payload["options"]["temperature"] = float(temperature)
    if top_p is not None:
        payload["options"] = payload.get("options", {})
        payload["options"]["top_p"] = float(top_p)

    t0 = time.time()
    with requests.post(url, json=payload, stream=True) as r:
        r.raise_for_status()
        out = []
        for line in r.iter_lines():
            if not line:
                continue
            data = json.loads(line.decode("utf-8"))
            if "response" in data:
                out.append(data["response"])
            if data.get("done"):
                break
    dt = time.time() - t0
    return "".join(out), dt

prompt = "Summarize in one sentence: Large Language Models are changing how we build AI systems."
text, dt = generate_raw(MODEL, prompt, temperature=0.7, top_p=0.9)
print(text, "\n\n[latency_sec]", round(dt, 2))

row = {
    "scenario":"summarization_raw",
    "temperature":0.7,
    "top_p":0.9,
    "output_len":len(text.split()),
    "latency_sec":dt,
    "note":"/api/generate path with sampling"
}
pd.concat([pd.read_csv(LOG_PATH), pd.DataFrame([row])]).to_csv(LOG_PATH, index=False)


Large Language Models (LLMs) are revolutionizing the field of artificial intelligence by enabling more efficient, effective, and flexible approaches to building intelligent systems. 

[latency_sec] 4.87



## 8) 로그 확인


In [23]:

import pandas as pd
df = pd.read_csv(LOG_PATH)
df


,scenario,temperature,top_p,output_len,latency_sec,note
0,summarization,NaN,NaN,48,31.303983,2-3 sentence summary
1,qa,NaN,NaN,92,7.845136,LLM LifeCycle question
2,style_transfer,NaN,NaN,27,3.981948,casual rewrite
3,summarization_raw,0.7,0.9,22,2.206811,/api/generate path with sampling
4,summarization,NaN,NaN,49,13.276957,2-3 sentence summary
5,summarization,NaN,NaN,30,4.322007,2-3 sentence summary
6,qa,NaN,NaN,105,12.721062,LLM LifeCycle question
7,style_transfer,NaN,NaN,45,7.215989,casual rewrite
8,summarization_raw,0.7,0.9,23,4.869223,/api/generate path with sampling
